In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_signal(spectrum_bins, spectrum_pdf, rate, T_total=5e-6,
                    pulse_width=25e-9, dt=1e-9, gain=1.0):
    # 归一化谱
    pdf = spectrum_pdf / np.sum(spectrum_pdf)
    cdf = np.cumsum(pdf)
    bin_centers = (spectrum_bins[:-1] + spectrum_bins[1:]) / 2

    # 泊松过程生成到达时间
    t, times = 0.0, []
    while t < T_total:
        t += np.random.exponential(1.0 / rate)
        if t < T_total:
            times.append(t)
    times = np.array(times)

    # 根据谱抽样能量
    u = np.random.rand(len(times))
    idx = np.searchsorted(cdf, u)
    energies = bin_centers[np.clip(idx, 0, len(bin_centers) - 1)]
    amplitudes = gain * energies

    # 生成时间轴与信号
    t_axis = np.arange(0, T_total, dt)
    signal = np.zeros_like(t_axis)

    pulse_samples = int(pulse_width / dt)
    for ti, Ai in zip(times, amplitudes):
        i0 = int(ti / dt)
        i1 = min(i0 + pulse_samples, len(signal))
        signal[i0:i1] += Ai

    return t_axis, signal, times, amplitudes

# 示例：虚拟谱（两个峰）
bins = np.linspace(0, 100, 101)
pdf = 0.7*np.exp(-(bins[:-1]-30)**2/(2*5**2)) + 0.3*np.exp(-(bins[:-1]-70)**2/(2*8**2))

# 入射率 1e6 cps
t, sig, times, amps = generate_signal(bins, pdf, rate=1e6, T_total=5e-6)

plt.figure(figsize=(10,3))
plt.plot(t*1e6, sig, label='Input Pulses')
plt.axhline(25, color='r', linestyle='--', label='~25 keV Threshold')
plt.xlabel('Time (μs)')
plt.ylabel('Amplitude (arb.)')
plt.legend()
plt.tight_layout()
plt.show()
